## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import ast
import datetime
import json
from scipy.stats import mannwhitneyu
import random
import datetime
from datetime import datetime
import networkx as nx
from scipy.stats import mannwhitneyu
import statistics
from networkx.algorithms import assortativity
from scipy.signal import savgol_filter
import seaborn as sns
import os
from scipy import signal
import math
import string
import scipy
from scipy import stats


## Functions

In [ ]:
def fix_format_and_replace_pos_barcodes(sem_interactions,barcode_dict,filename):
    new_rows = []
    count = 0
    for row,val in sem_interactions.iterrows():
        interactions_list = val['interactions']
        if (pd.isna(interactions_list)):
            user = val['user']
            converted_user = user
            if user in barcode_dict.keys():
                converted_user = barcode_dict[user]
            to_add = [val['day'],converted_user,val['building'],float("nan"),float("nan")]
            new_rows.append(to_add)
        else:
            interactions_list = [interactions_list[1:-1]]
            for x in interactions_list:
                y = ast.literal_eval(x.replace('[','[\"').replace(',','",').replace('\'\",','\','))
                interactions = []
                if type(y)!=str:
                    for item in list(y):
                        new_int = ast.literal_eval(item)
                        id = new_int[0]
                        if id in barcode_dict.keys():
                            converted_id = barcode_dict[id]
                            new_int = [converted_id,new_int[1]]
                        interactions.append(new_int)
                else:
                    new_int = ast.literal_eval(y)
                    id = new_int[0]
                    if id in barcode_dict.keys():
                        converted_id = barcode_dict[id]
                        new_int = [converted_id,new_int[1]]
                    interactions.append(new_int)
            user = val['user']
            converted_user= user
            if user in barcode_dict.keys():
                converted_user = barcode_dict[user]
            prefix = [val['day'],converted_user,val['building']]
            for x in interactions:
                new_list = prefix + x
                to_add = new_list
                new_rows.append(to_add)
    interactions_full = pd.DataFrame(new_rows,columns = ['day','user_1','building','user_2','duration'])
    interactions_full.to_csv(filename,index = False)

def make_by_ap_cleaned_interaction_lists(sem_interactions,guest_list,barcode_dict,filename,non_students_list):
    sem_interactions = sem_interactions.drop(columns = ['start_ts','end_ts','interactions'])
    sem_interactions = sem_interactions[~sem_interactions['user'].isin(guest_list)]
    sem_interactions = sem_interactions[~sem_interactions['user'].isin(non_students_list)]

    new_rows = []
    for row, val in sem_interactions.iterrows():
        user = val['user']
        day = val['day']
        duration = val['duration']
        building = val['building']
        ap = val['ap']
        int_count = val['interaction_count']
        if user in barcode_dict.keys():
            user = barcode_dict[user]
        to_add = [day,user,duration,building,ap,int_count]
        new_rows.append(to_add)
    sem_interactions_final = pd.DataFrame(new_rows,columns = ['day','user','duration','building','ap','int_count'])
    sem_interactions_final.to_csv(filename,index = False)

# File Reads

In [ ]:
#read in all guest/student flag file
authentication_info = pd.read_csv('authenticated_student_file.csv', compression='gzip', header=0, sep=',', quotechar='"')

students = authentication_info[authentication_info['is_student'] == True]
students = list(students['identity'].values)

non_students = authentication_info[authentication_info['is_student'] == False]
non_students_list= list(non_students['identity'].values)

In [ ]:
#read in concatenated wifi data
cleaned_new_full = pd.read_csv('new_full_file.csv')

In [ ]:
#cut dates
cut_summer = cleaned_new_full[cleaned_new_full['day']>='2020-08-17']
cut_summer2 = cut_summer[cut_summer['day']<='2021-04-30']
cut_break = cut_summer2.drop(cut_summer2[((cut_summer2['day']<='2021-01-18') & (cut_summer2['day']>='2020-11-20'))].index)
new_cleandate_file = cut_break

In [ ]:
#pull in barcode key cmu 2 da
barcode_key = pd.read_csv('cmu_2_da.csv')

In [ ]:
#clean na barcodes
barcode_key_cleaned = barcode_key.dropna().reset_index()

#make barcode dict mapping user to cmu barcode
barcode_dict = {}
for row,val in barcode_key_cleaned.iterrows():
    barcode_dict[val['user']] = val['barcode']

#create guest list (ie.users that only came on campus 1-3 days during year valid dates
guest_list = []
for row, val in new_cleandate_file.groupby('user').count().iterrows():
    if val['day'] < 4:
        guest_list.append(row)

#create fall interactions file
fall_interactions = new_cleandate_file[new_cleandate_file['day'] >= '2020-08-17']
fall_interactions = fall_interactions[fall_interactions['day'] <= '2020-11-20']

#create spring interactions file
spring_interactions = new_cleandate_file[new_cleandate_file['day'] >= '2021-01-18']
spring_interactions = spring_interactions[spring_interactions['day'] <= '2021-04-30']

In [ ]:
#count all users precleaning
all_users = []
for row, val in new_cleandate_file.groupby('user').count().iterrows():
    all_users.append(row)

#count all users in fall semester precleaning
fall_users = []
for row, val in fall_interactions.groupby('user').count().iterrows():
    fall_users.append(row)

#count all users in spring semester precleaning
spring_users = []
for row, val in spring_interactions.groupby('user').count().iterrows():
    spring_users.append(row)


In [ ]:
#count all users before and after cleaning with no repeats of users by using sets
users_after_cleaning_new = set(all_users) - (set(non_students_list + guest_list))
print('all users before cleaning',len(all_users))
print('new_cleaning', len(users_after_cleaning_new))

#count fall users before and after cleaning with no repeats of users by using sets
users_after_cleaning_new = set(fall_users) - (set(non_students_list + guest_list))
print('fall users before cleaning',len(fall_users))
print('new user count fall', len(users_after_cleaning_new))


#count spring users before and after cleaning with no repeats of users by using sets
users_after_cleaning_new = set(spring_users) - (set(non_students_list + guest_list))
print('spring users before cleaning',len(spring_users))
print('new user count spring', len(users_after_cleaning_new))

In [ ]:
#removing solo ints
fall_interactions_no_solos = fall_interactions.dropna().reset_index()
spring_interactions_no_solos = spring_interactions.dropna().reset_index()

In [ ]:
#fix_format_and_replace_pos_barcodes(fall_interactions,barcode_dict,'fall_interactions_full.csv')
fall_interactions_full = pd.read_csv('fall_interactions_full.csv')

#fix_format_and_replace_pos_barcodes(spring_interactions,barcode_dict,'spring_interactions_full.csv')
spring_interactions_full = pd.read_csv('spring_interactions_full.csv')

In [ ]:
#print('running')
#make_by_ap_cleaned_interaction_lists(fall_interactions,guest_list,barcode_dict,'fall_interactions_by_ap_cleaned.csv',non_students_list)
#print('halfway')
#make_by_ap_cleaned_interaction_lists(spring_interactions,guest_list,barcode_dict,'spring_interactions_by_ap_cleaned.csv',non_students_list)
#print('just reading')
fall_interactions_by_ap = pd.read_csv('fall_interactions_by_ap_cleaned.csv')
spring_interactions_by_ap = pd.read_csv('spring_interactions_by_ap_cleaned.csv')

In [ ]:
#remove guests and stagant devices from spring and fall full un-duped interactions files
fall_interactions_cleaned = fall_interactions_full[~fall_interactions_full['user_1'].isin(guest_list)]
fall_interactions_cleaned = fall_interactions_cleaned[~fall_interactions_cleaned['user_2'].isin(guest_list)]
fall_interactions_cleaned = fall_interactions_cleaned[~fall_interactions_cleaned['user_1'].isin(non_students_list)]
fall_interactions_cleaned = fall_interactions_cleaned[~fall_interactions_cleaned['user_2'].isin(non_students_list)]

spring_interactions_cleaned = spring_interactions_full[~spring_interactions_full['user_1'].isin(guest_list)]
spring_interactions_cleaned = spring_interactions_cleaned[~spring_interactions_cleaned['user_2'].isin(guest_list)]
spring_interactions_cleaned = spring_interactions_cleaned[~spring_interactions_cleaned['user_1'].isin(non_students_list)]
spring_interactions_cleaned = spring_interactions_cleaned[~spring_interactions_cleaned['user_2'].isin(non_students_list)]

print(len(fall_interactions_cleaned),len(fall_interactions_full))
print(len(spring_interactions_cleaned),len(spring_interactions_full))

In [ ]:
metadata = pd.read_csv('metadata_all.csv')
sequenced = metadata[~metadata['Sequence_ID'].isnull()]
peacock = metadata[metadata['pango_lineage'] == 'B.1.429.1']

In [ ]:
#fall:
fall_interactions_cleaned_barcodes_user1s = list(fall_interactions_cleaned['user_1'].values)
fall_interactions_cleaned_barcodes_user2s = list(fall_interactions_cleaned['user_2'].values)
fall_interactions_cleaned_barcodes = list(set(fall_interactions_cleaned_barcodes_user1s + fall_interactions_cleaned_barcodes_user2s))
fall_interactions_cleaned_barcodes = [x for x in fall_interactions_cleaned_barcodes if pd.isna(x) == False]

#spring:
spring_interactions_cleaned_barcodes_user1s = list(spring_interactions_cleaned['user_1'].values)
spring_interactions_cleaned_barcodes_user2s = list(spring_interactions_cleaned['user_2'].values)
spring_interactions_cleaned_barcodes = list(set(spring_interactions_cleaned_barcodes_user1s + spring_interactions_cleaned_barcodes_user2s))
spring_interactions_cleaned_barcodes = [x for x in spring_interactions_cleaned_barcodes if pd.isna(x) == False]

sequenced_barcodes = sequenced.dropna(subset = ['barcode'])['barcode'].values
positive_barcodes = metadata.dropna(subset = ['barcode'])['barcode'].values

metadata_of_positives = metadata.dropna(subset = ['Class_Year'])
fall_metadata_of_positives = metadata_of_positives[metadata_of_positives['Test_day'] <= '2020-11-20']
fall_metadata_of_positives = fall_metadata_of_positives[fall_metadata_of_positives['Test_day'] >= '2020-08-17'].reset_index(drop = True)
fall_metadata_of_positives_barcodes = fall_metadata_of_positives.dropna(subset = ['barcode'])['barcode'].values

spring_metadata_of_positives = metadata_of_positives[metadata_of_positives['Test_day'] <= '2021-04-30']
spring_metadata_of_positives = spring_metadata_of_positives[spring_metadata_of_positives['Test_day'] >= '2021-01-18'].reset_index(drop = True)
spring_metadata_of_positives_barcodes = spring_metadata_of_positives.dropna(subset = ['barcode'])['barcode'].values

sequenced_with_classyear = sequenced.dropna(subset = ['Class_Year'])
sequenced_with_classyear = sequenced_with_classyear[sequenced_with_classyear['Class_Year'] != 'Year E']

fall_sequenced = sequenced_with_classyear[sequenced_with_classyear['Test Date'] <= '2020-11-20']
fall_sequenced = fall_sequenced[fall_sequenced['Test Date'] >= '2020-08-17'].reset_index(drop = True)
fall_sequenced_barcodes = fall_sequenced.dropna(subset = ['barcode'])['barcode'].values

spring_sequenced = sequenced_with_classyear[sequenced_with_classyear['Test Date'] <= '2021-04-30']
spring_sequenced = spring_sequenced[spring_sequenced['Test Date'] >= '2021-01-18'].reset_index(drop = True)
spring_sequenced_barcodes = spring_sequenced.dropna(subset = ['barcode'])['barcode'].values

fall_wifi_count_positives = []
fall_wifi_count_sequenced = []
fall_wifi_count_pos_and_seq = []

for user in fall_interactions_cleaned_barcodes:
    if (user in fall_metadata_of_positives_barcodes) and (user in fall_sequenced_barcodes):
        fall_wifi_count_pos_and_seq.append(user)
    if user in fall_sequenced_barcodes:
        fall_wifi_count_sequenced.append(user)
    if user in fall_metadata_of_positives_barcodes:
        fall_wifi_count_positives.append(user)


print('all fall seq regardless of wifi', len(fall_sequenced_barcodes))
print('all fall pos regardless of wifi', len(fall_metadata_of_positives_barcodes))
print('fall pos', len(fall_wifi_count_positives))
print('fall seq', len(fall_wifi_count_sequenced))
print('fall pos and seq', len(fall_wifi_count_pos_and_seq))
print()
print()

spring_wifi_count_positives = []
spring_wifi_count_sequenced = []
spring_wifi_count_pos_and_seq = []

for user in spring_interactions_cleaned_barcodes:
    if (user in spring_metadata_of_positives_barcodes) and (user in spring_sequenced_barcodes):
        spring_wifi_count_pos_and_seq.append(user)
    if user in spring_sequenced_barcodes:
        spring_wifi_count_sequenced.append(user)
    if user in spring_metadata_of_positives_barcodes:
        spring_wifi_count_positives.append(user)


print('all spring seq regardless of wifi', len(spring_sequenced_barcodes))
print('all spring pos regardless of wifi', len(spring_metadata_of_positives_barcodes))
print('spring pos', len(spring_wifi_count_positives))
print('spring seq', len(spring_wifi_count_sequenced))
print('spring pos and seq', len(spring_wifi_count_pos_and_seq))

## Dedup Functions

In [ ]:
def group_sem_no_solos(semester_interactions_cleaned):
    interactions_no_solos = semester_interactions_cleaned.dropna().reset_index()
    interactions_no_solos['pair'] = '[\"' + interactions_no_solos['user_1']+'\",\"'+interactions_no_solos['user_2'] +'\"]'
    interactions_no_solos['pair'] = [json.loads(x) for x in interactions_no_solos['pair']]
    interactions_no_solos['pair'] = [str(sorted(x)) for x in interactions_no_solos['pair']]
    interactions_no_solos = interactions_no_solos.reset_index(drop = True)
    interactions_no_solos = interactions_no_solos[interactions_no_solos['user_1'] != interactions_no_solos['user_2']]
    grouped_by_pair = interactions_no_solos.groupby(['day','pair','user_1']).agg({'duration':['sum','median','count']}).reset_index()
    grouped_by_pair.columns = ['day','pair','user1','duration_sum','duration_median','int_count']
    return grouped_by_pair,interactions_no_solos

def run_dedup_and_clean_fall(grouped_by_pair_fall,fall_interactions_no_solos):
    #producing only pairs to see if we have two for each pair (what we would expect)
    pair_counts = grouped_by_pair_fall.groupby(['day','pair']).size().reset_index()

    #finding any pairs that don't have 2 interactions when all interaction info is summed
    pair_counts.columns = ['day','pair','count']
    faulty_pair_counts = pair_counts[pair_counts['count'] != 2]

    new_holder_cleaned = grouped_by_pair_fall

    #spliting into two lists that create pairwise comparisons of the same pairings
    new_holder1 = new_holder_cleaned[::2]
    new_holder2 =  new_holder_cleaned[1::2]

    #checking pairwise matching after clean of wonky 1-ways
    vals_h1 = list(new_holder1['pair'].values)
    vals_h2 = list(new_holder2['pair'].values)
    print(vals_h1 == vals_h2)

    new_rows_deduped = []
    for x in range(len(new_holder1)):
        if new_holder1['int_count'].iloc[x] != new_holder2['int_count'].iloc[x]:
            if new_holder1['int_count'].iloc[x] > new_holder2['int_count'].iloc[x]:
                file = new_holder1.iloc[x]
            else : #new_holder1.iloc['int_count'][x] < new_holder2.iloc['int_count'][x]
                file = new_holder2.iloc[x]
        else: #new_holder1.iloc['int_count'][x] == new_holder2.iloc['int_count']
            file = new_holder1.iloc[x]
        date = file['day']
        pair = file['pair']
        sum = file['duration_sum']
        median = file['duration_median']
        count = file['int_count']
        to_add = [date,pair,sum,median,count]
        new_rows_deduped.append(to_add)
    final_semester_deduped = pd.DataFrame(data = new_rows_deduped,columns = ['day','pair','sum','median','count'])

    #checking to make sure half were removed
    print(len(final_semester_deduped))
    print(len(new_holder_cleaned))
    print(len(new_holder_cleaned)/2)

    return final_semester_deduped

def run_dedup_and_clean_spring(grouped_by_pair_spring,spring_interactions_no_solos):
    #producing only pairs to see if we have two for each pair (what we would expect)
    pair_counts = grouped_by_pair_spring.groupby(['day','pair']).size().reset_index()

    #finding any pairs that don't have 2 interactions when all interaction info is summed
    pair_counts.columns = ['day','pair','count']

    new_holder_cleaned = grouped_by_pair_spring

    #spliting into two lists that create pairwise comparisons of the same pairings
    new_holder1 = new_holder_cleaned[::2]
    new_holder2 =  new_holder_cleaned[1::2]

    #checking pairwise matching after clean of wonky 1-ways
    vals_h1 = list(new_holder1['pair'].values)
    vals_h2 = list(new_holder2['pair'].values)
    print(vals_h1 == vals_h2)

    new_rows_deduped = []
    for x in range(len(new_holder1)):
        if new_holder1['int_count'].iloc[x] != new_holder2['int_count'].iloc[x]:
            if new_holder1['int_count'].iloc[x] > new_holder2['int_count'].iloc[x]:
                file = new_holder1.iloc[x]
            else : #new_holder1.iloc['int_count'][x] < new_holder2.iloc['int_count'][x]
                file = new_holder2.iloc[x]
        else: #new_holder1.iloc['int_count'][x] == new_holder2.iloc['int_count']
            file = new_holder1.iloc[x]
        date = file['day']
        pair = file['pair']
        sum = file['duration_sum']
        median = file['duration_median']
        count = file['int_count']
        to_add = [date,pair,sum,median,count]
        new_rows_deduped.append(to_add)
    final_semester_deduped = pd.DataFrame(data = new_rows_deduped,columns = ['day','pair','sum','median','count'])

    #checking to make sure half were removed
    print(len(final_semester_deduped))
    print(len(new_holder_cleaned))
    print(len(new_holder_cleaned)/2)
    return final_semester_deduped
    
def create_by_date_pos_list(semester_final,barcode_to_test_date,window_size):
    barcode_to_test_date_dict = {}
    by_date_pos_list = {}
    days = list(set(semester_final['day']))
    positives = list(set(barcode_to_test_date['barcode']))
    for row,index in barcode_to_test_date.iterrows():
        barcode_to_test_date_dict[index['barcode']] = index['Test Date']
    for day in days:
        by_date_pos_list[day] = []
        for positive in positives:
            test_date = datetime.fromisoformat(barcode_to_test_date_dict[positive])
            current_date = datetime.fromisoformat(day)
            diff = (test_date-current_date).days
            if diff <=window_size and diff >= 0:
                by_date_pos_list[day].append(positive)
    return by_date_pos_list, positives

In [ ]:
barcode_to_test_date = metadata[['barcode','Test Date']]
barcode_to_symptom_onset = metadata[['barcode','Symptom Onset']] #not currently used

barcode_to_test_date_dict = pd.Series(barcode_to_test_date['Test Date'].values,index = barcode_to_test_date['barcode']).to_dict()
barcode_to_symptom_onset_dict = pd.Series(barcode_to_symptom_onset['Symptom Onset'].values,index = barcode_to_symptom_onset['barcode']).to_dict()

In [ ]:
#grouped_by_pair_fall,fall_interactions_no_solos = group_sem_no_solos(fall_interactions_cleaned)
#final_fall = run_dedup_and_clean_fall(grouped_by_pair_fall,fall_interactions_no_solos)
#final_fall['pair'] = [x.replace('\'','') for x in final_fall['pair']]
#final_fall['pair'] = [x.strip('][').split(', ') for x in final_fall['pair']]
#final_fall['user1'] = [x[0] for x in final_fall['pair']]
#final_fall['user2'] = [x[1] for x in final_fall['pair']]
#final_fall = final_fall.drop(columns = 'pair')
#final_fall = final_fall.reindex(columns = ['day','user1','user2','sum','median','count'])
#final_fall.to_csv('final_fall.csv',index = False)

final_fall = pd.read_csv('final_fall.csv')

In [ ]:
#grouped_by_pair_spring,spring_interactions_no_solos = group_sem_no_solos(spring_interactions_cleaned)
#final_spring = run_dedup_and_clean_spring(grouped_by_pair_spring,spring_interactions_no_solos)
#final_spring['pair'] = [x.replace('\'','') for x in final_spring['pair']]
#final_spring['pair'] = [x.strip('][').split(', ') for x in final_spring['pair']]
#final_spring['user1'] = [x[0] for x in final_spring['pair']]
#final_spring['user2'] = [x[1] for x in final_spring['pair']]
#final_spring = final_spring.drop(columns = 'pair')
#final_spring = final_spring.reindex(columns = ['day','user1','user2','sum','median','count'])
#final_spring.to_csv('final_spring.csv',index = False)


final_spring = pd.read_csv('final_spring.csv')


# Figure Info

In [ ]:
import matplotlib as mpl
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname='C:/Users/brync/anaconda3/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/Montserrat.ttf',
    name='Montserrat')
fm.fontManager.ttflist.insert(0, fe)
mpl.rcParams['font.family'] = fe.name

# Counts for Table

In [ ]:
by_date_pos_list_fall,positives = create_by_date_pos_list(final_fall,barcode_to_test_date,10)
by_date_pos_list_spring,positives = create_by_date_pos_list(final_spring,barcode_to_test_date,10)

In [ ]:
final_fall = final_fall.groupby(['day'])
final_fall = [group for x, group in final_fall]
final_spring = final_spring.groupby(['day'])
final_spring = [group for x, group in final_spring]

fall_interactions_cleaned = fall_interactions_cleaned.groupby(['day'])
fall_interactions_cleaned = [group for x, group in fall_interactions_cleaned]
spring_interactions_cleaned = spring_interactions_cleaned.groupby(['day'])
spring_interactions_cleaned = [group for x, group in spring_interactions_cleaned]

In [ ]:
def run_network_to_make_groups(interactions_by_pairing):
    days = []
    node_count = []
    edge_count = []
    node_list = {}
    for x in interactions_by_pairing:
        x.columns = ['day','user_1','user_2','sum','median','count']
        day = str(x['day'].iloc[0])
        days.append(day)
        g = nx.from_pandas_edgelist(x,'user_1','user_2', ['sum','median','count'], create_using = nx.Graph())
        for node in g:
            if node in node_list:
                node_list[node][1].append(1)
            else:
                node_list[node] = [[day],[1]]
        num_nodes = g.number_of_nodes()
        num_edges = g.number_of_edges()

        node_count.append(num_nodes)
        edge_count.append(num_edges)
    return node_list,node_count,edge_count,days

def run_network_to_make_groups_largerfile(interactions_by_pairing):
    days = []
    node_count = []
    edge_count = []
    node_list = {}
    for x in interactions_by_pairing:
        day = str(x['day'].iloc[0])
        days.append(day)
        g = nx.from_pandas_edgelist(x,'user_1','user_2', ['duration'], create_using = nx.Graph())
        for node in g:
            if node in node_list:
                node_list[node][1].append(1)
            else:
                node_list[node] = [[day],[1]]
        num_nodes = g.number_of_nodes()
        num_edges = g.number_of_edges()

        node_count.append(num_nodes)
        edge_count.append(num_edges)
    return node_list,node_count,edge_count,days

In [ ]:
fall_node_list, fall_node_count, fall_edge_count, days_fall = run_network_to_make_groups(final_fall)
spring_node_list, spring_node_count, spring_edge_count, days_spring = run_network_to_make_groups(final_spring)

In [ ]:
def create_by_date_pos_list_without10days(by_date_pos_list,sem_node_list,positives,barcode_to_test_date_dict):
    sem_remove = []
    for x in sem_node_list:
        if x in positives:
            first = datetime.fromisoformat(sem_node_list[x][0][0])
            second = datetime.strptime(barcode_to_test_date_dict[x],"%Y-%m-%d")
            diff = (first-second).days
            if (diff < 1 and diff > -10):
                sem_remove.append(x) 

    by_date_pos_list_dict_new = {}
    for x in by_date_pos_list:
        new_list = []
        for code in by_date_pos_list[x]:
            if code not in sem_remove:
                new_list.append(code)
        by_date_pos_list_dict_new[x] = new_list
    
    return by_date_pos_list_dict_new

def create_numvisits_per_node_chart_sem1(node_list_sem1,node_list_sem2):
    
    values = []
    for x in node_list_sem1.values():
        values.append(x[1])
    sum_visits_sem1 = []
    for x in values:
        sum_visits_sem1.append(sum(x))
        
    values = []
    for x in node_list_sem2.values():
        values.append(x[1])
    sum_visits_sem2 = []
    for x in values:
        sum_visits_sem2.append(sum(x))
    
    plt.figure(figsize = (10,8))
    
    plt.hist(sum_visits_sem1,bins = range(100),alpha = .6,color = color_pallet[3],label = 'Fall 2020')
    plt.hist(sum_visits_sem2,bins = range(100),alpha = .6, color = color_pallet[5], label = 'Spring 2021')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('Number of Days on Campus During Semester',fontsize = 16)
    plt.ylabel('Number of Users',fontsize = 16)
    plt.legend(loc = 'best', fontsize = 16,frameon = True)

    image_format = 'svg' # e.g .png, .svg, etc.
    image_name = 'Supp5B.svg'

    plt.savefig(image_name, format=image_format, dpi=1200)
    plt.show()

    return 

In [ ]:
fall_interactions_full = fall_interactions_full.groupby(['day'])
fall_interactions_full = [group for x, group in fall_interactions_full]
spring_interactions_full = spring_interactions_full.groupby(['day'])
spring_interactions_full = [group for x, group in spring_interactions_full]

In [ ]:
by_date_pos_list_fall_dict = create_by_date_pos_list_without10days(by_date_pos_list_fall,fall_node_list,positives,barcode_to_test_date_dict)  
by_date_pos_list_spring_dict = create_by_date_pos_list_without10days(by_date_pos_list_spring,spring_node_list,positives,barcode_to_test_date_dict)  

In [ ]:
peacock_list = list(peacock['barcode'])
sequenced_list = list(sequenced['barcode'])
positiveIDs = list(metadata['barcode'])

In [ ]:
all_year_positives = pd.DataFrame(columns = ['day','p1','p2','day sum duration','day median duration','day interaction count'])
pos_IDs = positiveIDs
print('running')

for j in final_fall:
    j.columns = ['day','s1','s2','sum','median','count']
    day = j['day'].iloc[0]
    print(day)
    g = nx.from_pandas_edgelist(j,'s1','s2',['sum','median','count'], create_using = nx.Graph())
    if str(day) in by_date_pos_list_fall_dict:
        rolling_positives = by_date_pos_list_fall_dict[str(day)]
    elif str(day) not in by_date_pos_list_fall_dict:
        by_date_pos_list_fall_dict[str(day)] = []
    
    rolling_positives = by_date_pos_list_fall_dict[str(day)]
    edges = list(g.edges(data = True))
    for x in edges:
        if (str(x[0]) in rolling_positives) and (str(x[0]) in pos_IDs):
            if (str(x[1]) in rolling_positives) and (str(x[1]) in pos_IDs):
                all_year_positives.loc[len(all_year_positives.index)] = [day,x[0],x[1],x[2]['sum'],x[2]['median'],x[2]['count']]

print('running2')

for j in final_spring:
    j.columns = ['day','s1','s2','sum','median','count']
    day = j['day'].iloc[0]
    print(day)
    g = nx.from_pandas_edgelist(j,'s1','s2',['sum','median','count'], create_using = nx.Graph())
    if str(day) in by_date_pos_list_spring_dict:
        rolling_positives = by_date_pos_list_spring_dict[str(day)]
    elif str(day) not in by_date_pos_list_spring_dict:
        by_date_pos_list_spring_dict[str(day)] = []
    
    rolling_positives = by_date_pos_list_spring_dict[str(day)]
    edges = list(g.edges(data = True))
    for x in edges:
        if (str(x[0]) in rolling_positives) and (str(x[0]) in pos_IDs):
            if (str(x[1]) in rolling_positives) and (str(x[1]) in pos_IDs):
                all_year_positives.loc[len(all_year_positives.index)] = [day,x[0],x[1],x[2]['sum'],x[2]['median'],x[2]['count']]


In [ ]:
all_year_pos_aggregated = all_year_positives.groupby(['p1','p2']).agg({'day sum duration':'sum','day median duration':'median','day interaction count':['sum','size']}).reset_index()
new_columns = [''.join(t) for t in all_year_pos_aggregated.columns]
all_year_pos_aggregated.columns = new_columns
all_year_pos_aggregated.columns = ['p1','p2','total interactions','median duration median','sum time spent','total days interacted']

In [ ]:
all_year_pos_aggregated['inverse total interaction time'] = 1/all_year_pos_aggregated['sum time spent']

In [ ]:
all_year_pos_aggregated.to_csv('all_year_pos_aggregated_refreshed.csv')

In [ ]:
all_year_pos_aggregated = all_year_pos_aggregated[all_year_pos_aggregated['p1'].isin(metadata['barcode'].values)]
all_year_pos_aggregated = all_year_pos_aggregated[all_year_pos_aggregated['p2'].isin(metadata['barcode'].values)]

In [ ]:
all_all_graph = nx.from_pandas_edgelist(all_year_pos_aggregated,'p1','p2',['sum time spent','median duration median', 'total interactions', 'total days interacted','inverse total interaction time'],create_using = nx.Graph())
weight = nx.get_edge_attributes(all_all_graph,'inverse total interaction time')
weight = list(weight.values())
weight = [x/(max(weight))*2 for x in weight]
#weight = .5
plt.figure(figsize = (20,20))
color_map = []
node_color_dict = {}
peacock_nodes = []
non_peacock_nodes = []
for node in all_all_graph:
    if node in peacock_list:
        node_color_dict[node] = 'orange'
        color_map.append('orange')
        peacock_nodes.append(node)
    elif node in sequenced_list:
        node_color_dict[node] = 'skyblue'
        color_map.append('skyblue')
        non_peacock_nodes.append(node)
    else: 
        node_color_dict[node] = 'black'
        color_map.append('black')   

nodes_to_hold = all_all_graph.nodes

nx.set_node_attributes(all_all_graph,node_color_dict,'sequence_status')


node_list_dict_peacock = {}
node_list_dict_nonpeacock = {}

peacock_dist = []
non_peacock_dist = []
for node in all_all_graph:
    for node2 in all_all_graph:
        if node != node2:
            if nx.has_path(all_all_graph,node,node2):
                if (str([node,node2]) not in node_list_dict_peacock) and (str([node2,node]) not in node_list_dict_peacock) and (str([node,node2]) not in node_list_dict_nonpeacock) and (str([node2,node]) not in node_list_dict_nonpeacock):
                    if node in peacock_list and node2 in peacock_list:
                        node_list_dict_peacock[str([node,node2])] = nx.shortest_path_length(all_all_graph,node,node2)
                    elif (node not in peacock_list and node2 in peacock_list) or (node in peacock_list and node2 not in peacock_list):
                        node_list_dict_nonpeacock[str([node,node2])] = nx.shortest_path_length(all_all_graph,node,node2)

print(len(node_list_dict_peacock))
print(len(node_list_dict_nonpeacock))




nx.draw(all_all_graph,pos =nx.spring_layout(all_all_graph,seed = 4,k=0.089, iterations=45),width = weight,with_labels = False,node_size = 100,alpha = .6,linewidths = .9,edge_color = 'grey',node_color = color_map,font_size = 14)
#nx.draw(all_all_graph,pos =nx.kamada_kawai_layout(all_all_graph,scale = 4),width = weight,with_labels = False,node_size = 100,alpha = .6,linewidths = .9,edge_color = 'royalblue',node_color = color_map,font_size = 14)

plt.show()

In [ ]:

barcode_2_lineage = {}
for x in range(len(sequenced['barcode'])):
    barcode_2_lineage[sequenced['barcode'].iloc[x]] = sequenced['pango_lineage'].iloc[x]
    
{key:val for key, val in barcode_2_lineage.items() if val != None}
{key:val for key, val in barcode_2_lineage.items() if val != np.nan}

for node in all_all_graph:
    if node not in barcode_2_lineage:
        barcode_2_lineage[node] = 'NOT SEQUENCED'


nx.set_node_attributes(all_all_graph,barcode_2_lineage,'sequence_lineage')
pos_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(all_all_graph, 'sequence_lineage')
print(pos_attribute_assort_coef)


plt.figure(figsize = (20,20))
nx.draw(all_all_graph,pos =nx.spring_layout(all_all_graph,seed = 27),with_labels = False,width = 5,node_size = 100,alpha = .6,linewidths = .9,edge_color = 'grey',node_color = color_map,font_size = 14)
plt.show()


print(all_all_graph)

In [ ]:
new_lineage_names = nx.get_node_attributes(all_all_graph, "sequence_lineage")
new_lineage_names = {key:str(value).replace('nan','None').replace('NOT SEQUENCED','None') for (key,value) in new_lineage_names.items()}
nx.set_node_attributes(all_all_graph, new_lineage_names, name="sequence_lineage")
graph_attributes = list(set(new_lineage_names.values()))

#[[PL],[MinPL,MaxPl,MedPL] [C, S,KT,KA] 
# KT = percent total, KA = percent found attributes, S = structure size, c = count, PL = path length
all_node_details = []
num_levels = 3
total_in_structure = 0
for node in all_all_graph.nodes(data = True):
    node_details = {node[0]:{i:[[],[],[],[]] for i in graph_attributes}}
    node_details[node[0]][node[1]['sequence_lineage']][0].append(0)
    actual_lineage = node[1]['sequence_lineage']
    for counter in range(1,num_levels+1):
        level = nx.descendants_at_distance(all_all_graph,node[0],counter)
        total_in_structure += len(level)+1
        for secondary_node in all_all_graph.nodes(data = True):
            if secondary_node[0] in level:
                node_details[node[0]][secondary_node[1]['sequence_lineage']][0].append(counter)
    for attribute in node_details[node[0]]:
        node_details[node[0]][attribute][1]= [min(node_details[node[0]][attribute][0],default = np.nan),max(node_details[node[0]][attribute][0],default = np.nan)]
        if len(node_details[node[0]][attribute][0]) == 0:
            node_details[node[0]][attribute][1].append(0)
        else:
            node_details[node[0]][attribute][1].append(float(sum(node_details[node[0]][attribute][0])/len(node_details[node[0]][attribute][0])))
        node_details[node[0]][attribute][2].append(len(node_details[node[0]][attribute][0]))
        node_details[node[0]][attribute][2].append(total_in_structure)
        node_details[node[0]][attribute][2].append(abs((len(node_details[node[0]][attribute][0]))/(total_in_structure)))
        if (total_in_structure - len(node_details[node[0]]['None'][0])) != 0:
            node_details[node[0]][attribute][2].append(abs(len(node_details[node[0]][attribute][0])/((total_in_structure) - len(node_details[node[0]]['None'][0]))))
        else:
            node_details[node[0]][attribute][2].append(0.0)
        node_details[node[0]][attribute][3].append(actual_lineage)
    node_details[node[0]]['None'][2][3]=0.0
    all_node_details.append(node_details)



new_frames = []
for node_details in all_node_details:
    for x in node_details:
        for y in list(node_details[x]):
            node = x
            lineage = y
            path_dists = node_details[x][y][0]
            min_path_dist = node_details[x][y][1][0]
            max_path_dist = node_details[x][y][1][1]
            median_path_dist = node_details[x][y][1][2]
            saturation = node_details[x][y][2][0]
            structure_size = node_details[x][y][2][1]
            percent_total = node_details[x][y][2][2]
            percent_known = node_details[x][y][2][3]
            actual_lineage = node_details[x][y][3][0]
            new_frames.append([node,lineage,path_dists,min_path_dist,max_path_dist,median_path_dist,saturation,structure_size,percent_total,percent_known,actual_lineage])
        
decision_frame = pd.DataFrame(data = new_frames, columns = ['node','lineage','path dists','min path dist','max path dist','avg path dist','saturation','structure size','percent total','percent known','actual_lineage'])

print(decision_frame)
decision_frames_by_node = decision_frame.groupby(['node'])
new_decision_frames = []
for x in decision_frames_by_node:
    decision_list = x[1]
    new_decision_frames.append([decision_list.dropna()])


In [ ]:
for x in new_decision_frames:
    dataframe = x[0]
    dataframe = dataframe.reset_index(drop = True)
    dataframe['lineage presence on campus'] = 'to be calculated'
    dataframe['level considered to'] = num_levels
    dataframe['Test Date'] = barcode_to_test_date_dict[dataframe['node'][0]]
    dataframe['Symptom Onset'] = barcode_to_symptom_onset_dict[dataframe['node'][0]]
    dataframe['Wastewater Presence Binary'] = 'to be calculated'
    dataframe['Wastewater Presence Proportion'] = 'to be calculated'
    print(dataframe)
    break

#to do, path dist with test date and symptom onset date for network structure
#not strict neighborhood size, dynamic for each individual based on likelihood of transmission 
#   ie. (stop searching subnetwork if it is just not feasible anymore)
#look at differences in time for each node in the subnetworks to define feasibility
#   ie. this will avoid choosing depth and prepositive window statically
#add mean field approx and metadata about node

In [ ]:
metadata.columns

In [ ]:
mavilys = metadata[['barcode','Mavily']]
mavilys = mavilys.dropna()
mavilys_dict = dict(mavilys.values)

sports = metadata[['barcode','Sports_team']]
sports = sports.dropna()
sports_dict = dict(sports.values)

residency = metadata[['barcode','Lives_on_campus']]
residency = residency.dropna()
residency_dict = dict(residency.values)

residence_hall = metadata[['barcode','Residence_hall']]
residence_hall = residence_hall.dropna()
residence_hall_dict = dict(residence_hall.values)

gender = metadata[['barcode','Sex']]
gender = gender.dropna()
gender_dict = dict(gender.values)

class_year = metadata[['barcode','Class_Year']].dropna()
class_year_dict = dict(class_year.values)



In [ ]:
def pos_att_network_sem_test_rolling(by_date_all_interactions_bypairing,positives_list_rolling):
    days = []
    class_year = []
    gender = []
    residence_hall = []
    residency = []
    sports = []
    mavilys = []
    
    for x in by_date_all_interactions_bypairing:
        day = str(x['day'].iloc[0])
        positives = positives_list_rolling[str(day)]
        days.append(day)
        x = x[x['s1'].isin(positives)]
        x = x[x['s2'].isin(positives)]
        g = nx.from_pandas_edgelist(x,'s1','s2',['sum','median', 'count'], create_using = nx.Graph())
        for nodes in g:
            if nodes in list(class_year_dict.keys()): 
                g.nodes[nodes]['class_year'] = class_year_dict[nodes]
            else: 
                g.nodes[nodes]['class_year'] = 0
            if nodes in list(gender_dict.keys()): 
                g.nodes[nodes]['gender'] = gender_dict[nodes]
            else: 
                g.nodes[nodes]['gender'] = 0
            if nodes in list(residence_hall_dict.keys()): 
                g.nodes[nodes]['residence_hall'] = residence_hall_dict[nodes]
            else: 
                g.nodes[nodes]['residence_hall'] = 0
            if nodes in list(residency_dict.keys()): 
                g.nodes[nodes]['residency'] = residency_dict[nodes]
            else: 
                g.nodes[nodes]['residency'] = 0
            if nodes in list(sports_dict.keys()): 
                g.nodes[nodes]['sports'] = sports_dict[nodes]
            else: 
                g.nodes[nodes]['sports'] = 0
            if nodes in list(mavilys_dict.keys()): 
                g.nodes[nodes]['mavilys'] = mavilys_dict[nodes]
            else: 
                g.nodes[nodes]['mavilys'] = 0

        class_year_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'class_year')
        gender_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'gender')
        res_hall_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'residence_hall')
        residency_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'residency')
        sports_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'sports')
        mavilys_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'mavilys')
        
        class_year.append(class_year_attribute_assort_coef)
        gender.append(gender_attribute_assort_coef)
        residence_hall.append(res_hall_attribute_assort_coef)
        residency.append(residency_attribute_assort_coef)
        sports.append(sports_attribute_assort_coef)
        mavilys.append(mavilys_attribute_assort_coef)
      
    return class_year,gender,residence_hall,residency,sports,mavilys,days

def pos_att_network_sem_test(by_date_all_interactions_bypairing,positives_list):
    days = []
    class_year = []
    gender = []
    residence_hall = []
    residency = []
    sports = []
    mavilys = []
    
    for x in by_date_all_interactions_bypairing:
        day = str(x['day'].iloc[0])
        positives = list(positives_list)
        days.append(day)
        x = x[x['s1'].isin(positives)]
        x = x[x['s2'].isin(positives)]
        g = nx.from_pandas_edgelist(x,'s1','s2',['sum','median', 'count'], create_using = nx.Graph())
        for nodes in g:
            if nodes in list(class_year_dict.keys()): 
                g.nodes[nodes]['class_year'] = class_year_dict[nodes]
            else: 
                g.nodes[nodes]['class_year'] = 0
            if nodes in list(gender_dict.keys()): 
                g.nodes[nodes]['gender'] = gender_dict[nodes]
            else: 
                g.nodes[nodes]['gender'] = 0
            if nodes in list(residence_hall_dict.keys()): 
                g.nodes[nodes]['residence_hall'] = residence_hall_dict[nodes]
            else: 
                g.nodes[nodes]['residence_hall'] = 0
            if nodes in list(residency_dict.keys()): 
                g.nodes[nodes]['residency'] = residency_dict[nodes]
            else: 
                g.nodes[nodes]['residency'] = 0
            if nodes in list(sports_dict.keys()): 
                g.nodes[nodes]['sports'] = sports_dict[nodes]
            else: 
                g.nodes[nodes]['sports'] = 0
            if nodes in list(mavilys_dict.keys()): 
                g.nodes[nodes]['mavilys'] = mavilys_dict[nodes]
            else: 
                g.nodes[nodes]['mavilys'] = 0

        class_year_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'class_year')
        gender_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'gender')
        res_hall_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'residence_hall')
        residency_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'residency')
        sports_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'sports')
        mavilys_attribute_assort_coef = assortativity.attribute_assortativity_coefficient(g, 'mavilys')
        
        class_year.append(class_year_attribute_assort_coef)
        gender.append(gender_attribute_assort_coef)
        residence_hall.append(res_hall_attribute_assort_coef)
        residency.append(residency_attribute_assort_coef)
        sports.append(sports_attribute_assort_coef)
        mavilys.append(mavilys_attribute_assort_coef)
      
    return class_year,gender,residence_hall,residency,sports,mavilys,days

def make_AA_charts(days,pos1,type_name1,pos2,type_name2,pos3,type_name3,pos4,type_name4,pos5,type_name5,pos6,type_name6):
    formatted_dates = []
    not_formatted_dates = []
    for x in days:
        new = datetime.strptime(x,'%Y-%m-%d')
        not_formatted_dates.append(new)
        new_time = datetime.strftime(new, '%b-%d')
        formatted_dates.append(new_time)
    
    plt.figure(figsize = (19,10))
    plt.plot(formatted_dates,pos1, color = 'blue',label = type_name1, linewidth = 3.5)
    plt.plot(formatted_dates,pos2, color = 'orange',label = type_name2, linewidth = 3.5)
    plt.plot(formatted_dates,pos3, color = 'seagreen',label = type_name3, linewidth = 3.5)
    plt.plot(formatted_dates,pos4, color = 'mediumpurple',label = type_name4, linewidth = 3.5)
    plt.plot(formatted_dates,pos5, color = 'yellowgreen',label = type_name5, linewidth = 3.5)
    plt.plot(formatted_dates,pos6, color = 'salmon',label = type_name6, linewidth = 3.5)
    plt.legend(loc = 'best',fontsize = 16)

    plt.xticks(np.arange(0, len(formatted_dates), 7),fontsize = 20,rotation = 70)
    plt.yticks(fontsize = 20)
    plt.ylabel('Assortativity Coefficient',fontsize = 20)
    plt.xlabel('Fall 2020',fontsize = 20)
    plt.show()

In [ ]:

class_year,gender,residence_hall,residency,sports,mavilys,days = pos_att_network_sem_test(final_fall,fall_metadata_of_positives_barcodes)
make_AA_charts(days,class_year,'Class Year',gender,'Gender',residence_hall,'Residence Hall',residency,'On/Off Campus Residency',sports,'Primary Sports Team',mavilys,'Mavily')

In [ ]:
class_year,gender,residence_hall,residency,sports,mavilys,days = pos_att_network_sem_test_rolling(final_fall,by_date_pos_list_fall_dict)

make_AA_charts(days,class_year,'Class Year',gender,'Gender',residence_hall,'Residence Hall',residency,'On/Off Campus Residency',sports,'Primary Sports Team',mavilys,'Mavily')

In [ ]:
days = pd.date_range(start='2020-08-17',end='2021-04-30')
days = [str(x.strftime('%Y-%m-%d')) for x in days]

for_prevalence = metadata[['pango_lineage','Test Date','Symptom Onset']]
for_prevalence = for_prevalence.dropna(subset = ['pango_lineage'])
for_prevalence = for_prevalence[for_prevalence['pango_lineage'] != 'None']
for_prevalence['date'] = [for_prevalence['Symptom Onset'].iloc[x] if (for_prevalence['Symptom Onset'].iloc[x] != 'asymptomatic' and for_prevalence['Symptom Onset'].iloc[x] != 'unknown') else for_prevalence['Test Date'].iloc[x] for x in range(len(for_prevalence['Test Date']))]
for_prevalence = for_prevalence.drop(columns = ['Test Date','Symptom Onset'])

all_lineages = for_prevalence['pango_lineage'].unique()
for_prevalence['count_of_sequenced'] = 1

counting = for_prevalence.groupby(['date','pango_lineage']).count().reset_index(drop = False)

days_holder = days.copy()
new_vals = []
for x in counting.groupby(['date']):
    new_df = x[1]
    holder_lineages = all_lineages
    found_lineages = list(new_df['pango_lineage'].values)
    needed_lineages = [x for x in holder_lineages if x not in found_lineages]
    for x in needed_lineages:
        new_row = [new_df['date'].iloc[0],x,0]
        new_df.loc[len(new_df.index)] = new_row
    string_v = new_df['date'].iloc[0]
    if string_v in days_holder:
        days_holder.remove(string_v)
    new_vals.append(new_df)
new_vals_full = pd.concat(new_vals)
new_vals_full = new_vals_full.reset_index(drop = True)

days_holder2 = days_holder
while days_holder != []:
    for x in days_holder:
        holder_lineages = all_lineages.copy()
        for y in holder_lineages:
            new_row = [x,y,0]
            new_vals_full.loc[len(new_vals_full.index)] = new_row
        string_v = x
        if string_v in days_holder:
            days_holder.remove(string_v)
            
new_vals_full = new_vals_full.sort_values(by='date')
new_vals_full['date'] = [datetime.strptime(x,'%Y-%m-%d') for x in new_vals_full['date']]

In [ ]:
rows_to_be_added = []
to_be_expanded = new_vals_full[new_vals_full['count_of_sequenced'] > 1].reset_index(drop = True)
vals_full = new_vals_full[new_vals_full['count_of_sequenced']<=1].reset_index(drop = True)
for index,row in to_be_expanded.iterrows():
    for x in range(row[2]):
        rows_to_be_added.append(list(row))

for x in rows_to_be_added:
    vals_full.loc[len(vals_full.index)] = x
    
vals_full = vals_full.drop(columns = ['count_of_sequenced'])

In [ ]:
vals_full = vals_full.sort_values(by = 'date')

plt.figure(figsize=(20,15))
sns.histplot(data = vals_full, x = 'date', hue = 'pango_lineage',common_norm = True,alpha = .8,kde = True,stat = 'probability',multiple = 'layer',shrink = .5,fill = False,palette = 'nipy_spectral')
plt.xticks(rotation = 90)
plt.show()

plt.figure(figsize=(20,15))
sns.kdeplot(data = vals_full, x = 'date', hue = 'pango_lineage',alpha = .1,multiple = 'layer',fill = False,palette = 'nipy_spectral',shade = True)
plt.xticks(rotation = 90)
plt.show()


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
np.random.seed(1234)  # Seed random generator
dens = sm.nonparametric.KDEUnivariate(vals_full[vals_full['pango_lineage'] == 'B.1.2'])
dens.fit()
plt.plot(dens.cdf)
plt.show()

In [ ]:
for_prevalence = for_prevalence.sort_values(by = 'date')
#for_prevalence['date'] = [datetime.strptime(x,'%Y-%m-%d') for x in for_prevalence['date']]
plt.figure(figsize=(20,15))
sns.histplot(data = for_prevalence, x = 'date', hue = 'pango_lineage',alpha = .8,kde = True,stat = 'probability',multiple = 'layer',shrink = .5,fill = False,palette = 'nipy_spectral')
plt.xticks(rotation = 90)
plt.show()

plt.figure(figsize=(20,15))
sns.kdeplot(data = for_prevalence, x = 'date', hue = 'pango_lineage',alpha = .15,multiple = 'layer',common_norm = True,fill = False,palette = 'nipy_spectral',shade = True)
plt.xticks(rotation = 90)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity
densities = []
Xs = []
for j in all_lineages:
    dataframe = new_vals_full[new_vals_full['pango_lineage'] == j]
    X=dataframe.loc[:,'count_of_sequenced'].values[:, np.newaxis]


    kde = KernelDensity(kernel='gaussian').fit(X)  
    log_density_values=kde.score_samples(X)
    density=np.exp(log_density_values)
    densities.append(density)
    Xs.append(X)


In [ ]:
colors = ['red','orange','yellow','green','blue','purple','violet','gold','hotpink','olive']
counter = 0
new = []
for x in range(len(densities)):
    new_d = [abs(y - max(densities[x])) for y in densities[x]]
    new.append(new_d)
plt.figure(figsize = (15,15))
for x in new:
    plt.plot(x,alpha = .5)
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse


# Multiplicative Decomposition 
result_mul = seasonal_decompose(for_prevalence['value'], model='multiplicative', extrapolate_trend='freq')

# Additive Decomposition
result_add = seasonal_decompose(df['value'], model='additive', extrapolate_trend='freq')

# Plot
plt.rcParams.update({'figure.figsize': (10,10)})
result_mul.plot().suptitle('Multiplicative Decompose', fontsize=22)
result_add.plot().suptitle('Additive Decompose', fontsize=22)
plt.show()

In [ ]:
for x in days:
    if x not in for_prevalence['date']:
        print(x)

In [ ]:
new_lineage_names = nx.get_node_attributes(all_all_graph, "sequence_lineage")
new_lineage_names = {key:str(value).replace('nan','None').replace('NOT SEQUENCED','None') for (key,value) in new_lineage_names.items()}
nx.set_node_attributes(all_all_graph, new_lineage_names, name="sequence_lineage")
graph_attributes = list(set(new_lineage_names.values()))

#[[PL],[MinPL,MaxPl,MedPL] [C, S,KT,KA] 
# KT = percent total, KA = percent found attributes, S = structure size, c = count, PL = path length
all_node_details = []
num_levels = 1
total_in_structure = 0
for node in all_all_graph.nodes(data = True):
    node_details = {node[0]:{i:[[],[],[],[]] for i in graph_attributes}}
    node_details[node[0]][node[1]['sequence_lineage']][0].append(0)
    actual_lineage = node[1]['sequence_lineage']
    for counter in range(1,num_levels+1):
        level = nx.descendants_at_distance(all_all_graph,node[0],counter)
        total_in_structure += len(level)+1
        for secondary_node in all_all_graph.nodes(data = True):
            if secondary_node[0] in level:
                test_date_node = barcode_to_test_date_dict[node[0]]
                test_date_secondary_node =  barcode_to_test_date_dict[secondary_node[0]]
                symp_date_node = barcode_to_symptom_onset_dict[node[0]]
                symp_date_secondary_node = barcode_to_symptom_onset_dict[secondary_node[0]] 
                if symp_date_node == 'asymptomatic' or symp_date_node == 'unknown':
                    node_date = test_date_node
                else:
                    node_date = symp_date_node
                if symp_date_secondary_node == 'asymptomatic' or symp_date_secondary_node == 'unknown':
                    secondary_node_date = test_date_secondary_node
                else:
                    secondary_node_date = symp_date_secondary_node
                
                days_diff = (datetime.strptime(node_date, '%Y-%m-%d') - datetime.strptime(secondary_node_date, '%Y-%m-%d')).days
                if days_diff < 10:
                    #check if interaction is in this time span
                    node_details[node[0]][secondary_node[1]['sequence_lineage']][0].append(counter)
    for attribute in node_details[node[0]]:
        node_details[node[0]][attribute][1]= [min(node_details[node[0]][attribute][0],default = np.nan),max(node_details[node[0]][attribute][0],default = np.nan)]
        if len(node_details[node[0]][attribute][0]) == 0:
            node_details[node[0]][attribute][1].append(0)
        else:
            node_details[node[0]][attribute][1].append(float(sum(node_details[node[0]][attribute][0])/len(node_details[node[0]][attribute][0])))
        node_details[node[0]][attribute][2].append(len(node_details[node[0]][attribute][0]))
        node_details[node[0]][attribute][2].append(total_in_structure)
        node_details[node[0]][attribute][2].append(abs((len(node_details[node[0]][attribute][0]))/(total_in_structure)))
        if (total_in_structure - len(node_details[node[0]]['None'][0])) != 0:
            node_details[node[0]][attribute][2].append(abs(len(node_details[node[0]][attribute][0])/((total_in_structure) - len(node_details[node[0]]['None'][0]))))
        else:
            node_details[node[0]][attribute][2].append(0.0)
        node_details[node[0]][attribute][3].append(actual_lineage)
    node_details[node[0]]['None'][2][3]=0.0
    all_node_details.append(node_details)



new_frames = []
for node_details in all_node_details:
    for x in node_details:
        for y in list(node_details[x]):
            node = x
            lineage = y
            path_dists = node_details[x][y][0]
            min_path_dist = node_details[x][y][1][0]
            max_path_dist = node_details[x][y][1][1]
            median_path_dist = node_details[x][y][1][2]
            saturation = node_details[x][y][2][0]
            structure_size = node_details[x][y][2][1]
            percent_total = node_details[x][y][2][2]
            percent_known = node_details[x][y][2][3]
            actual_lineage = node_details[x][y][3][0]
            new_frames.append([node,lineage,path_dists,min_path_dist,max_path_dist,median_path_dist,saturation,structure_size,percent_total,percent_known,actual_lineage])
        
decision_frame = pd.DataFrame(data = new_frames, columns = ['node','lineage','path dists','min path dist','max path dist','avg path dist','saturation','structure size','percent total','percent known','actual_lineage'])

print(decision_frame)
decision_frames_by_node = decision_frame.groupby(['node'])
new_decision_frames = []
for x in decision_frames_by_node:
    decision_list = x[1]
    new_decision_frames.append([decision_list.dropna()])

In [ ]:
for x in new_decision_frames:
    dataframe = x[0]
    dataframe = dataframe.reset_index(drop = True)
    dataframe['lineage presence on campus'] = 'to be calculated'
    dataframe['level considered to'] = num_levels
    dataframe['Test Date'] = barcode_to_test_date_dict[dataframe['node'][0]]
    dataframe['Symptom Onset'] = barcode_to_symptom_onset_dict[dataframe['node'][0]]
    dataframe['Wastewater Presence Binary'] = 'to be calculated'
    dataframe['Wastewater Presence Proportion'] = 'to be calculated'
    print(dataframe)
    break


In [ ]:
peacock_IDs = list(peacock['barcode'].values)

In [ ]:
genetic_distance = pd.read_csv('genetic_distance_cleaned_refreshed.csv')
path_info = pd.read_csv('new_full_cleaned_refreshed.csv',usecols = ['p1','p2','path dist','weighted path dist'])
genetic_distance.columns = ['index','p1','p2','snps']
path_info.columns = ['p1','p2','path dist','weighted path dist']
new_gen_dist = pd.merge(genetic_distance,path_info,on=['p1','p2'],how = 'left')
new_gen_dist.to_csv('path_and_gen_dist_refreshed.csv')


In [ ]:
final_cleaned_paths = pd.read_csv('path_and_gen_dist_refreshed.csv',usecols = ['p1','p2','snps','path dist','weighted path dist'])

In [ ]:
final_cleaned_paths['path_inf_or_not'] = ['infinite' if final_cleaned_paths['path dist'][x] == np.inf else 'finite' for x in range(len(final_cleaned_paths))]

In [ ]:
final_cleaned_paths['p1 status'] = ['peacock' if final_cleaned_paths['p1'][x] in peacock_IDs else 'non-peacock' for x in range(len(final_cleaned_paths))]
final_cleaned_paths['p2 status'] = ['peacock' if final_cleaned_paths['p2'][x] in peacock_IDs else 'non-peacock' for x in range(len(final_cleaned_paths))]

In [ ]:
final_cleaned_paths['peacock'] = ['peacock pair' if (final_cleaned_paths['p1 status'][x] == 'peacock' and final_cleaned_paths['p2 status'][x] == 'peacock') else 'non-peacock pair' for x in range(len(final_cleaned_paths))]

In [ ]:
barcode_2_lineage = {}
for x in range(len(sequenced['barcode'])):
    barcode_2_lineage[sequenced['barcode'].iloc[x]] = sequenced['pango_lineage'].iloc[x]
    
{key:val for key, val in barcode_2_lineage.items() if val != None}
{key:val for key, val in barcode_2_lineage.items() if val != np.nan}
{key:val for key, val in barcode_2_lineage.items() if val in nodes_to_hold}


In [ ]:
final_cleaned_paths['status'] = final_cleaned_paths['p1 status']+","+final_cleaned_paths['p2 status']
final_cleaned_paths['status'] = [len(list(set(x.split(',')))) for x in final_cleaned_paths['status']]
final_cleaned_paths = final_cleaned_paths[final_cleaned_paths['p2'].isin(nodes_to_hold)]
final_cleaned_paths = final_cleaned_paths[final_cleaned_paths['p1'].isin(nodes_to_hold)]


non_peacock_pairings = final_cleaned_paths[final_cleaned_paths['peacock'] == 'non-peacock pair' ]
non_peacock_pairings = non_peacock_pairings[non_peacock_pairings['status'] == 1 ]

peacock_pairs_df = final_cleaned_paths[final_cleaned_paths['peacock'] == 'peacock pair' ]

#make box plot for inf versus non inf for peacock versus non peacock
non_peacock_pairings_inf_snps = list(non_peacock_pairings[non_peacock_pairings['path_inf_or_not'] == 'infinite']['snps'])
non_peacock_pairings_fin_snps = list(non_peacock_pairings[non_peacock_pairings['path_inf_or_not'] == 'finite']['snps'])

peacock_pairings_inf_snps = list(peacock_pairs_df[peacock_pairs_df['path_inf_or_not'] == 'infinite']['snps'])
peacock_pairings_fin_snps = list(peacock_pairs_df[peacock_pairs_df['path_inf_or_not'] == 'finite']['snps'])


non_peacock_pairings_fin = non_peacock_pairings[non_peacock_pairings['path_inf_or_not'] == 'finite']
peacock_pairings_fin = peacock_pairs_df[peacock_pairs_df['path_inf_or_not'] == 'finite']


print(scipy.stats.mannwhitneyu(non_peacock_pairings_inf_snps,non_peacock_pairings_fin_snps))
print(scipy.stats.mannwhitneyu(peacock_pairings_inf_snps,peacock_pairings_fin_snps))


print(scipy.stats.mannwhitneyu(non_peacock_pairings_fin['path dist'],peacock_pairings_fin['path dist']))

non_peacock_pairings_fin['type'] = 1
peacock_pairings_fin['type'] = 0


for_dist_compare = pd.concat([non_peacock_pairings_fin,peacock_pairings_fin], ignore_index=True)



In [ ]:

u,p = scipy.stats.mannwhitneyu(peacock_pairings_inf_snps,peacock_pairings_fin_snps)
print(p,u)

medians = final_cleaned_paths[final_cleaned_paths['peacock'] == 'peacock pair' ].groupby(['path_inf_or_not'])['snps'].median().values
nobs = final_cleaned_paths[final_cleaned_paths['peacock'] == 'peacock pair' ]['path_inf_or_not'].value_counts().values
nobs = [str(x) for x in nobs.tolist()]
nobs = ["n: " + i for i in nobs]
 
print(medians)
print(nobs)




#make box plot for inf versus non inf for peacock versus non peacock
non_peacock_pairings_inf = list(non_peacock_pairings[non_peacock_pairings['path_inf_or_not'] == 'infinite']['snps'])
non_peacock_pairings_fin = list(non_peacock_pairings[non_peacock_pairings['path_inf_or_not'] == 'finite']['snps'])

peacock_pairings_inf = list(peacock_pairs_df[peacock_pairs_df['path_inf_or_not'] == 'infinite']['snps'])
peacock_pairings_fin = list(peacock_pairs_df[peacock_pairs_df['path_inf_or_not'] == 'finite']['snps'])

print(scipy.stats.mannwhitneyu(non_peacock_pairings_inf,non_peacock_pairings_fin))
print(scipy.stats.mannwhitneyu(peacock_pairings_inf,peacock_pairings_fin))

In [ ]:
inf_gen_dist_path_dist = final_cleaned_paths[final_cleaned_paths['path_inf_or_not']=='infinite']
non_inf_gen_dist_path_dist = final_cleaned_paths[final_cleaned_paths['path_inf_or_not']=='finite']

In [ ]:
print()
print('correlation path, snps, non infinite (all) ')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist['path dist'], b=non_inf_gen_dist_path_dist['snps']))

print()
print('correlation weighted path, snps, non infinite (all) ')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist['weighted path dist'], b=non_inf_gen_dist_path_dist['snps']))


print()
print('correlation path, snps, non infinite (all) snps < 40')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<40]['path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<40]['snps']))

print()
print('correlation weighted path, snps, non infinite (all) snps < 40')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<40]['weighted path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<40]['snps']))


print()
print('correlation path, snps, non infinite (all) snps < 30')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<30]['path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<30]['snps']))

print()
print('correlation weighted path, snps, non infinite (all) snps < 30')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<30]['weighted path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<30]['snps']))


print()
print('correlation path, snps, non infinite (all) snps < 25')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<25]['path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<25]['snps']))

print()
print('correlation weighted path, snps, non infinite (all) snps < 25')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<25]['weighted path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<25]['snps']))


print()
print('correlation path, snps, non infinite (all) snps < 20')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<20]['path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<20]['snps']))

print()
print('correlation weighted path, snps, non infinite (all) snps < 20')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<20]['weighted path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<20]['snps']))


print()
print('correlation path, snps, non infinite (all) snps < 15')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<15]['path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<15]['snps']))

print()
print('correlation weighted path, snps, non infinite (all) snps < 15')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<15]['weighted path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<15]['snps']))


print()
print('correlation path, snps, non infinite (all) snps < 10')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<10]['path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<10]['snps']))

print()
print('correlation weighted path, snps, non infinite (all) snps < 10')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<10]['weighted path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<10]['snps']))

print()
print('correlation path, snps, non infinite (all) snps < 5')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<5]['path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<5]['snps']))

print()
print('correlation weighted path, snps, non infinite (all) snps < 5')
print(scipy.stats.spearmanr(non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<5]['weighted path dist'], b=non_inf_gen_dist_path_dist[non_inf_gen_dist_path_dist['snps']<5]['snps']))


In [ ]:
print('running')
pe_pe_sum = []
pe_n_sum = []
pe_pe_median = []
pe_n_median = []
pe_pe_ints = []
pe_n_ints = []
pe_p_sum = []
pe_p_median = []
pe_p_ints = []

for j in final_spring:
    j.columns = ['day','s1','s2','sum duration','median duration','num interactions']
    day = j['day'].iloc[0]
    g = nx.from_pandas_edgelist(j,'s1','s2',['sum duration','median duration','num interactions'], create_using = nx.Graph())
    rolling_positives = by_date_pos_list_spring_dict[str(day)]
    edges = list(g.edges(data = True))
    for x in edges:
        if (str(x[0]) in rolling_positives) and (str(x[0]) in peacock_IDs): 
            if (str(x[1]) in rolling_positives) and (str(x[1]) in peacock_IDs): #pea - pea
                pe_pe_sum.append(x[2]['sum duration'])
                pe_pe_median.append(x[2]['median duration'])
                pe_pe_ints.append(x[2]['num interactions'])
            elif (str(x[1]) in rolling_positives) and (str(x[1]) not in peacock_IDs): #pea - p
                pe_p_sum.append(x[2]['sum duration'])
                pe_p_median.append(x[2]['median duration'])
                pe_p_ints.append(x[2]['num interactions'])
            else:                                                               #pea - n
                pe_n_sum.append(x[2]['sum duration'])
                pe_n_median.append(x[2]['median duration'])
                pe_n_ints.append(x[2]['num interactions'])
        if (str(x[1]) in rolling_positives) and (str(x[1]) in peacock_IDs): 
            if (str(x[0]) in rolling_positives) and (str(x[0]) not in peacock_IDs): #p - pea
                pe_p_sum.append(x[2]['sum duration'])
                pe_p_median.append(x[2]['median duration'])
                pe_p_ints.append(x[2]['num interactions'])
            elif (str(x[0]) not in rolling_positives) and (str(x[0]) not in peacock_IDs):   #n - pea
                pe_n_sum.append(x[2]['sum duration'])
                pe_n_median.append(x[2]['median duration'])
                pe_n_ints.append(x[2]['num interactions'])
        
